<a href="https://colab.research.google.com/github/gabPetti/CVision/blob/main/CVision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# CVision

![cv.jpg](https://imagens.ne10.uol.com.br/veiculos/_midias/jpg/2022/05/11/adriano_imperador_e_comando_vermelho-21001806.jpg)

## 🚀 Visão Geral

O CVision é uma ferramenta de PLN que atua como um consultor de RH digital. Em vez de uma análise de currículo genérica, este projeto foca em resolver um problema do mundo real que toda pessoa que já entrou no mercado de trabalho já se perguntou: "Meu currículo está bom para esta vaga?"

O usuário fornece dois inputs de texto:
- O texto completo do seu Currículo (CV).
- O texto completo da Descrição da Vaga (ex: copiado do LinkedIn, Vagas.com).

A ferramenta, então, gera uma análise de aderência (Gap Analysis) tática, informando os pontos fortes, as lacunas (o que falta) e sugestões estratégicas para o candidato se destacar. Além disso, o CVision pode **gerar um novo currículo otimizado em HTML e exportá-lo para PDF**, incorporando essas sugestões para maximizar as chances do candidato.

## ✨ Features
- **Gap Analysis**: Compara o CV do usuário com os requisitos da vaga.
- **Extração de Requisitos**: Identifica e extrai automaticamente as hard skills e soft skills mais importantes da descrição da vaga.
- **Relatório Tático**: Gera um relatório simples em markdown com:
  - Pontos Fortes: Onde o seu CV brilha para esta vaga.
  - Pontos de Melhoria: Quais requisitos da vaga não estão claros no seu CV.
  - Sugestão Estratégica: Uma dica de ouro para destacar na sua carta de apresentação ou entrevista.
- **Geração de CV Otimizado**: Cria um currículo em formato HTML, incorporando as informações existentes e as sugestões estratégicas para melhor adequação à vaga.
- **Exportação para PDF**: Converte o currículo HTML otimizado em um arquivo PDF para fácil compartilhamento e impressão.

## 🛠️ Arquitetura e Uso Criativo do LangChain

Aqui está o núcleo do projeto, utilizando o LangChain e múltiplas técnicas de PLN para alcançar as features descritas acima.

Em vez de um prompt único, o projeto usa a LCEL (LangChain Expression Language) para orquestrar um pipeline de duas etapas que simula o raciocínio de um recrutador. A pipeline consiste na utilização das tecnicas de Extração de Entidades encadeado de Sumarização Comparativa.

Cadeia 1: Extração de Requisitos
- Input: O `texto_vaga`.
- Processo: O texto passa por um `PromptTemplate` que instrui o LLM a atuar como um "Tech Recruiter".
- Técnica: O LLM (Gemini) extrai as skills essenciais.
- Output: Um `JsonOutputParser` força o LLM a retornar um JSON estruturado com as `hard_skills` e `soft_skills`.

Cadeia 2: Análise de Aderência
- Input: O texto_cv do usuário + o JSON gerado pela Cadeia 1.
- Processo: Um PromptTemplate instrui o LLM a atuar como "Consultor de Carreira".
- Técnica: O LLM (Gemini) recebe ambos os inputs e deve comparar os dois, gerando a análise de aderência (gap analysis).
- Output: Um `StrOutputParser` retorna o relatório final em texto (Markdown).

Cadeia 3: Geração de Currículo Otimizado
- Input: O `cv_resumido` e o `resultado` da análise de aderência.
- Processo: Um PromptTemplate instrui o LLM a atuar como "Gerador de CV em HTML", utilizando as informações fornecidas e as sugestões estratégicas para criar um HTML bem estruturado.
- Técnica: O LLM (Gemini) gera o código HTML completo do currículo.
- Output: Um `StrOutputParser` retorna o código HTML.

O RunnablePassthrough do LangChain é usado para gerenciar e rotear esses múltiplos inputs (CV, Vaga) através do pipeline de forma eficiente.

## ✅ Atendimento aos Critérios de Avaliação
- Uso do LangChain: Uso da LCEL para orquestrar um pipeline sequencial, gerenciar múltiplos inputs e usar parsers.
- Uso de um LLM: Uso do Gemini 2.5 Flash para ambas as etapas de PLN.
- Uso de Página Web (Corpus): O corpus é o texto de uma vaga de emprego real (ex: LinkedIn, Vagas.com), um dado não-estruturado do "mundo real" que o usuário fornece.
- Github: O projeto está disponível [link-para-seu-github-aqui].
- Criatividade: A criatividade reside na arquitetura de duas etapas, que simula o raciocínio de um recrutador (primeiro entende a vaga, depois analisa o CV), ao invés de usar um prompt único e simplista. Este projeto exala criatividade e inovação


## 🔧 Tecnologias Utilizadas
- Linguagem: Python
- Plataforma: Google Colab
- Framework: LangChain (LCEL, langchain-google-genai, langchain-core)
- LLM: Google Gemini 2.5 Flash
- Conversão HTML para PDF: WeasyPrint

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

### 1. Instação de dependencias

In [28]:
!pip install langchain-google-genai langchain-community pypdf ipython weasyprint

### 2. Instanciação e configuração

In [29]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.prompts import ChatPromptTemplate
from IPython.display import display, Markdown
from langchain_core.runnables import RunnablePassthrough
from weasyprint import HTML, CSS
from google.colab import userdata, files
from weasyprint.text.fonts import FontConfiguration
import json

In [30]:
GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
llm = ChatGoogleGenerativeAI(
    temperature = 0,
    model = "gemini-2.5-flash",
    api_key = GOOGLE_API_KEY
)
parser_json = JsonOutputParser()
parser_str = StrOutputParser()
link_vaga = "https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4210992740"

# Carregar PDF
file_path = "./CV.pdf"
loader = PyPDFLoader(file_path)

pages = loader.load_and_split()

### 3. Sumarização de CV

In [31]:
prompt_sumarizacao = ChatPromptTemplate.from_template("""
Extraia e organize as informações do currículo abaixo em uma tabela ou lista limpa. Ignore informações irrelevantes ou textos genéricos.

Estruture a resposta assim:
    Cargos/Título Principais:
    Tempo de Experiência: (Calcule o tempo total baseado nas datas)
    Empresas Anteriores: (Liste apenas os nomes)
    Educação:
    Projetos:
    Idiomas:
    Stack Tecnológico/Ferramentas:

Currículo: {cv}
""")

chain_sumarizacao = prompt_sumarizacao | llm | parser_str

cv_resumido = chain_sumarizacao.invoke({
    "cv": pages
})

display(Markdown(cv_resumido))

Aqui está a extração e organização das informações do currículo:

**Cargos/Título Principais:**
*   Estagiário, Desenvolvedor Web Full Stack
*   Freelancer, Desenvolvedor Web

**Tempo de Experiência:**
*   Estagiário (Ago 2024 — Presente): 1 ano e 3 meses (considerando a data de criação do CV em Nov 2025 como "Presente")
*   Freelancer (Jul 2023 — Out 2023): 4 meses
*   **Tempo Total:** Aproximadamente 1 ano e 7 meses

**Empresas Anteriores:**
*   Limoney
*   Conexus

**Educação:**
*   Duplo Bacharelado em Ciência e Tecnologia e Ciência da Computação, Universidade Federal do ABC (Set 2022 — Set 2027)

**Projetos:**
*   Rede Social (desenvolvida como Freelancer para Conexus, com Front End em React e Back End em Node/Express com MongoDB)
*   Análise IMDb (liderou análise de dataset IMDb usando teoria dos grafos com Python, Polars, NetworkX e visualização com Gephi)
*   Bot do Discord (bot simples em Python com pipeline CI/CD automático usando AWS EC2)

**Idiomas:**
*   Português (nativo)
*   Inglês (fluente/proficiente C2, pontuação EF SET 77/100)

**Stack Tecnológico/Ferramentas:**
*   **Linguagens:** HTML, CSS, TypeScript, C++, Python, JavaScript
*   **Frameworks/Bibliotecas:** Nest.js, Node.js, React, Redux, JQuery, Tailwind CSS, Express, Tanstack Query, Polars, NetworkX
*   **Bancos de Dados:** MongoDB, PostgreSQL
*   **Cloud/DevOps:** AWS (Lambda, S3, MSK, SQS, SNS, EC2), Docker, Git, NPM, CI/CD
*   **Conceitos/APIs:** APIs REST, Open Finance, Cloud
*   **Ferramentas:** Gephi

### 4. Análise de Aderência

In [32]:
prompt_analise = ChatPromptTemplate.from_template("""
Você é um 'Consultor de Carreira'.
Faça uma análise de aderência (Gap Analysis) em Markdown comparando o CV do candidato com os dados encontrados da vaga.

Seja tático e gere um relatório com:
- **Pontos Fortes**: Onde o CV atende aos requisitos.
- **Pontos de Melhoria**: Quais requisitos da vaga faltam no CV.
- **Sugestão Estratégica**: Uma dica de ouro para o CV.

**Link da Vaga (JSON):**
{link_vaga}

**Currículo (CV) do Candidato:**
{cv}
""")

chain_analise = prompt_analise | llm | parser_str

resultado = chain_analise.invoke({
    "link_vaga": link_vaga,
    "cv": cv_resumido
})

print("\n--- RELATÓRIO DE ANÁLISE DE ADERÊNCIA ---")
display(Markdown(resultado))


--- RELATÓRIO DE ANÁLISE DE ADERÊNCIA ---


Como seu Consultor de Carreira, realizei uma análise detalhada do seu currículo em comparação com os requisitos típicos de uma vaga de Desenvolvedor Full Stack, considerando o contexto de uma vaga no LinkedIn para a qual você estaria se candidatando.

**Observação Importante:** Como o link da vaga fornecido (`https://www.linkedin.com/jobs/collections/recommended/?currentJobId=4210992740`) aponta para uma coleção de vagas recomendadas e não para uma vaga específica com um JSON detalhado, a análise abaixo é baseada em um perfil de vaga de Desenvolvedor Full Stack (nível Júnior/Pleno) comumente encontrado no mercado, que valoriza as tecnologias e experiências que você possui.

---

## Relatório de Análise de Aderência (Gap Analysis)

### Perfil do Candidato:
*   **Cargos/Título Principais:** Estagiário, Desenvolvedor Web Full Stack; Freelancer, Desenvolvedor Web
*   **Tempo de Experiência:** ~1 ano e 7 meses (Estagiário + Freelancer)
*   **Educação:** Duplo Bacharelado em Ciência e Tecnologia e Ciência da Computação (UFABC, previsão 2027)
*   **Stack Tecnológico:** HTML, CSS, TypeScript, C++, Python, JavaScript, Nest.js, Node.js, React, Redux, JQuery, Tailwind CSS, Express, Tanstack Query, Polars, NetworkX, MongoDB, PostgreSQL, AWS (Lambda, S3, MSK, SQS, SNS, EC2), Docker, Git, NPM, CI/CD, APIs REST, Open Finance, Cloud, Gephi.
*   **Idiomas:** Português (nativo), Inglês (fluente/proficiente C2).
*   **Projetos:** Rede Social (React, Node/Express, MongoDB), Análise IMDb (Python, teoria dos grafos), Bot do Discord (Python, AWS EC2, CI/CD).

---

### Pontos Fortes (Onde o CV atende aos requisitos)

Seu currículo demonstra uma aderência excepcional em diversas áreas-chave para um Desenvolvedor Full Stack:

1.  **Domínio Tecnológico Abrangente e Atualizado:**
    *   **Full Stack Robusto:** Você possui uma stack completa e moderna, cobrindo tanto o frontend (React, Redux, TypeScript, Tailwind CSS, Tanstack Query) quanto o backend (Node.js, Express, Nest.js, Python).
    *   **Bancos de Dados:** Experiência com MongoDB (NoSQL) e PostgreSQL (SQL), mostrando versatilidade.
    *   **Conceitos Essenciais:** Sólido entendimento de APIs REST e arquitetura de nuvem.
    *   **Linguagens Diversas:** Além de JavaScript/TypeScript, você tem Python e C++, o que indica uma base de programação forte e adaptabilidade.

2.  **Experiência Sólida em Cloud e DevOps:**
    *   **AWS Hands-on:** Sua experiência com múltiplos serviços AWS (Lambda, S3, MSK, SQS, SNS, EC2) é um diferencial enorme, especialmente para vagas que buscam engenheiros com mentalidade de nuvem.
    *   **Práticas de DevOps:** Conhecimento em Docker, Git e CI/CD demonstra que você entende e aplica boas práticas de desenvolvimento e entrega contínua.

3.  **Projetos Relevantes e Diversificados:**
    *   **Aplicação Prática:** Os projetos listados (Rede Social, Análise IMDb, Bot do Discord) não são apenas teóricos; eles mostram sua capacidade de construir soluções completas, desde o desenvolvimento web até análise de dados e automação com CI/CD.
    *   **Iniciativa e Liderança:** Liderar a análise do dataset IMDb e desenvolver um bot com pipeline CI/CD são exemplos claros de proatividade e habilidades de engenharia.

4.  **Proficiência em Inglês:**
    *   Seu nível C2 comprovado (EF SET 77/100) é um grande trunfo, abrindo portas para empresas multinacionais e equipes distribuídas.

5.  **Base Acadêmica Forte:**
    *   O Duplo Bacharelado em Ciência e Tecnologia e Ciência da Computação em uma universidade federal renomada indica uma base teórica sólida e capacidade de aprendizado contínuo.

---

### Pontos de Melhoria (Quais requisitos da vaga faltam no CV)

Estes são os pontos onde seu currículo pode não atender totalmente aos requisitos de vagas de nível mais elevado ou onde há espaço para fortalecer sua candidatura:

1.  **Tempo de Experiência Profissional Formal:**
    *   Com aproximadamente 1 ano e 7 meses de experiência total (incluindo estágio e freelancer), você se encaixa perfeitamente em vagas de **Estágio Avançado, Júnior ou Trainee**.
    *   Para vagas de **Desenvolvedor Pleno** (que geralmente pedem 2-5 anos de experiência) ou Sênior, seu tempo de experiência formal pode ser percebido como um "gap". Embora seus projetos e stack sejam fortes, a experiência em um ambiente corporativo em tempo integral é um fator que muitas empresas consideram.

2.  **Conclusão da Graduação:**
    *   A previsão de conclusão em 2027 significa que você ainda está cursando a faculdade. Para vagas que exigem um diploma de bacharel *concluído*, isso seria um impedimento. No entanto, para vagas de estágio ou júnior, é perfeitamente aceitável.

3.  **Experiência em Ambientes Corporativos Maiores/Estruturados:**
    *   Embora a experiência freelancer seja valiosa, algumas empresas podem dar mais peso à experiência em tempo integral em equipes maiores, com processos de desenvolvimento mais formais, code reviews extensos e sistemas legados. Seu CV não detalha o tamanho das equipes ou a complexidade dos ambientes de trabalho anteriores.

---

### Sugestão Estratégica (Uma dica de ouro para o CV)

Dada a sua impressionante base técnica e projetos, a sugestão estratégica mais valiosa é:

**"Posicione-se estrategicamente para vagas de nível Júnior/Trainee ou Estágio Avançado, e, ao se candidatar a posições de nível Pleno, construa uma narrativa proativa em seu CV e carta de apresentação que demonstre como seus projetos e habilidades técnicas avançadas compensam a experiência formal, quantificando o impacto e as responsabilidades assumidas em cada experiência."**

**Como aplicar esta dica:**

*   **Para Vagas Júnior/Trainee/Estágio Avançado:** Seu CV já é muito forte. Apenas certifique-se de que a descrição da vaga se alinha com o que você busca e que você pode demonstrar entusiasmo para aprender e crescer.
*   **Para Vagas Pleno (e acima):**
    *   **Reestruture a Seção de Experiência:** Para cada cargo (Estagiário, Freelancer), vá além das tarefas. Descreva os *desafios* que você enfrentou, as *soluções* que implementou e os *resultados quantificáveis* que alcançou. Ex: "Otimizei a performance da API em X%, resultando em uma redução de Y segundos no tempo de resposta."
    *   **Destaque a Autonomia e Liderança:** Se você teve autonomia em projetos freelancer ou liderou iniciativas como estagiário, enfatize isso. Mostre que você não apenas executa, mas também pensa estrategicamente e toma decisões.
    *   **Portfólio Online Robusto:** Crie um portfólio online (site pessoal, GitHub bem organizado) que sirva como uma extensão do seu CV. Inclua demos, links para repositórios e descrições detalhadas dos seus projetos, explicando as tecnologias usadas, os desafios e suas contribuições. Isso é crucial para compensar a falta de experiência formal.
    *   **Carta de Apresentação Personalizada:** Para cada vaga de nível Pleno, escreva uma carta de apresentação que aborde diretamente a questão da experiência. Explique como, apesar do tempo formal, sua profundidade técnica, projetos complexos e experiência com Cloud/DevOps o preparam para as responsabilidades da vaga.

Seu perfil é extremamente promissor. Com um posicionamento estratégico e uma comunicação eficaz de suas conquistas, você tem um futuro brilhante na área de desenvolvimento!

### 5. Geração de CV

In [49]:
prompt_html_cv = ChatPromptTemplate.from_template("""
Você é um 'Consultor de Carreira'. Sua tarefa é gerar um currículo profissional em formato HTML, incorporando as informações do currículo e aplicando as sugestões estratégicas para otimizá-lo para a vaga.

Certifique-se de que o HTML seja bem-estruturado, e replique exatamente o estilo do currículo e inclua todas as seções relevantes do currículo. Priorize a clareza e a facilidade de leitura.

A estilização do HTML deve ser feita inline nos elementos. Utilize flex box ou grid se necessário para garantir que o estilo do cv original seja replicado da forma mais fiel.

**Currículo do Candidato:**
{cv}

**Sugestões Estratégicas e Análise de Aderência:**
{resultado}

Por favor, forneça apenas o código HTML completo, sem qualquer texto adicional ou explicação. O HTML deve ser pronto para uso.
""")

chain_html_cv = prompt_html_cv | llm | parser_str

html_cv = chain_html_cv.invoke({
    "cv": pages,
    "resultado": resultado
}).replace('```html\n', '').replace('\n```', '')

print("HTML CV gerado com sucesso!")

HTML CV gerado com sucesso!


In [51]:
font_config = FontConfiguration()
html = HTML(string=html_cv)
# css = CSS(string='''body{background:light-blue!important;font-family:-apple-system,BlinkMacSystemFont,"Segoe UI",Roboto,"Helvetica Neue",Arial,sans-serif;color:#333;line-height:1.6;background-color:#fff;max-width:800px;margin:0 auto;padding:40px}header{text-align:center;padding-bottom:20px;margin-bottom:30px;border-bottom:2px solid #f0f0f0}header h1{font-size:2.6rem;font-weight:700;color:#000;margin-bottom:10px}header p{font-size:0.95rem;color:#555;line-height:1.5}a{color:#0056b3;text-decoration:none}a:hover{text-decoration:underline}section{margin-bottom:25px}h2{font-size:1.5rem;font-weight:600;color:#222;border-bottom:1px solid #ccc;padding-bottom:5px;margin-bottom:20px}h3{font-size:1.2rem;font-weight:600;color:#111;margin-bottom:3px}article{margin-bottom:20px}article > p{font-size:1rem;font-style:italic;color:#555;margin-bottom:15px}ul{list-style-type:disc;margin-left:20px;padding-left:15px}li{font-size:0.95rem;margin-bottom:8px}li > strong,p > strong{font-weight:600;color:#000}''', font_config=font_config)

html.write_pdf('cv_otimizado.pdf')

print("PDF 'cv_otimizado.pdf' gerado com sucesso!")

DEBUG:fontTools.ttLib.ttFont:Reading 'maxp' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'maxp' table
DEBUG:fontTools.subset.timer:Took 0.003s to load 'maxp'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'maxp'
INFO:fontTools.subset:maxp pruned
DEBUG:fontTools.ttLib.ttFont:Reading 'cmap' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'cmap' table
DEBUG:fontTools.ttLib.ttFont:Reading 'post' table from disk
DEBUG:fontTools.ttLib.ttFont:Decompiling 'post' table
DEBUG:fontTools.subset.timer:Took 0.006s to load 'cmap'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'cmap'
INFO:fontTools.subset:cmap pruned
INFO:fontTools.subset:fpgm dropped
INFO:fontTools.subset:prep dropped
INFO:fontTools.subset:cvt  dropped
INFO:fontTools.subset:kern dropped
DEBUG:fontTools.subset.timer:Took 0.000s to load 'post'
DEBUG:fontTools.subset.timer:Took 0.000s to prune 'post'
INFO:fontTools.subset:post pruned
INFO:fontTools.subset:GPOS dropped
INFO:fontTools.subset:GSUB dropped
DEBUG:f

PDF 'cv_otimizado.pdf' gerado com sucesso!


In [52]:
files.download('cv_otimizado.pdf')
print("Download do PDF 'cv_otimizado.pdf' iniciado!")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Download do PDF 'cv_otimizado.pdf' iniciado!
